In [34]:
import pandas as pd 
import numpy as np
import public_timeseries_testing_util as optiver2023
from torch.nn.utils.rnn import pack_padded_sequence, pack_sequence, unpack_sequence, unpad_sequence
import torch
from tqdm.notebook import trange,tqdm
import torch.nn as nn 
import torch.optim as optim
import wandb
import torch_classes
from model_saver import model_saver_wandb as model_saver
import training_testing
import importlib

In [2]:
env = optiver2023.make_env()
iter_test = env.iter_test()

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [387]:
model = torch_classes.GRUNet(12,64)
model_loc = f"C:/Users/Nick/Documents/GitHub/OptiverKaggle/models/tough-totem-105/tough-totem-105_80.pt"
model_data = torch.load(model_loc,map_location=torch.device('cpu'))
print(model_data['model_state_dict'].keys())
model.load_state_dict(model_data['model_state_dict'], strict=False)

odict_keys(['gru.weight_ih_l0', 'gru.weight_hh_l0', 'gru.bias_ih_l0', 'gru.bias_hh_l0', 'batch_norm.weight', 'batch_norm.bias', 'batch_norm.running_mean', 'batch_norm.running_var', 'batch_norm.num_batches_tracked', 'fc0.weight', 'fc0.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'fc3.weight', 'fc3.bias'])


<All keys matched successfully>

In [364]:
importlib.reload(torch_classes)

<module 'torch_classes' from 'c:\\Users\\Nick\\Documents\\GitHub\\OptiverKaggle\\torch_classes.py'>

In [5]:
stats_cols =  ['seconds_in_bucket','imbalance_size','imbalance_buy_sell_flag','reference_price','matched_size','far_price','near_price','bid_price','bid_size','ask_price','ask_size','wap']

In [6]:
trading_data = torch_classes.TradingData()

In [388]:
trading_data.fill_hidden_states_for_test(model_data['db_train'])
trading_data.reset_hidden(64,device='cpu')

0
tensor([[ 0.4570, -0.9641,  0.0556,  0.3187, -0.8199,  0.2145, -0.6593, -0.4434,
         -0.4239, -0.3600, -0.0571,  0.9848, -0.3532,  0.9210, -0.3768, -0.5132,
          0.8861, -0.1514, -0.9260, -0.8000, -0.9793, -1.0000,  0.3105, -0.9878,
          0.8955,  0.3536, -0.6440,  0.1685, -0.9328, -0.9824, -0.7731,  0.9151,
         -0.0879, -0.9859,  0.9996,  0.5933,  0.2711, -0.8528,  0.3635,  0.7888,
         -0.8207,  0.3336,  0.8360, -0.5640, -0.7918, -0.7105,  0.8392,  0.9965,
         -0.9069, -0.5620,  0.8810, -0.8549, -0.5570, -0.9973, -0.9536,  0.9862,
         -1.0000,  0.5586, -0.8086,  0.8054, -0.9664,  0.5031, -0.1587,  0.9859]])
1
tensor([[ 0.5573, -0.9996, -0.0875, -0.8667, -0.8092,  0.1909, -0.0873, -0.1651,
         -0.1603, -0.0932, -0.3419,  0.9190,  0.8175,  0.9407, -0.1747, -0.1076,
          0.4734, -0.7020,  0.0317, -0.9356,  0.1350, -1.0000,  0.6581, -0.4255,
          0.9169,  0.3826,  0.7131,  0.6376, -0.5815, -0.4303, -0.9984,  0.2914,
          0.8451, -0.3

In [8]:
def gen_preds(test, trading_data:torch_classes.TradingData, model:torch_classes.GRUNet):
    test['stats']  = pd.Series(test[stats_cols].fillna(-1).values.tolist())
    stock_ids = test.stock_id.unique().tolist()
    stocks = [trading_data.stocksDict[x] for x in stock_ids]
    hidden = torch.stack([trading_data.stocksDict[x].hidden for x in stock_ids]).transpose(0,1)
    
    X = [torch.tensor(x) for x in test['stats'].tolist()]
    
    Xstacked = torch.stack(X)
    
    Xviewed = pack_sequence(Xstacked.view(-1,1,12))
    
    output,hidden_out = model(Xviewed,hidden)
    
    hidden_out = hidden_out.transpose(0,1)
    
    [setattr(obj, 'hidden', val.detach()) for obj, val in zip(stocks,hidden_out)]
    
    output = output.flatten().tolist()
    
    return output

In [9]:
counter = 0
for (test, revealed_targets, sample_prediction) in iter_test:
    if counter == 0:
        print(test.head(3))
        print(revealed_targets.head(3))
        print(sample_prediction.head(3))
        
    preditcions = gen_preds(test,trading_data,model)
    
    sample_prediction['target'] = preditcions
    test_df = test
    # print(preditcions)
    break
    env.predict(sample_prediction)
    counter += 1

   stock_id  date_id  seconds_in_bucket  imbalance_size  \
0         0        0                  0      3180602.69   
1         1        0                  0       166603.91   
2         2        0                  0       302879.87   

   imbalance_buy_sell_flag  reference_price  matched_size  far_price  \
0                        1         0.999812   13380276.64        NaN   
1                       -1         0.999896    1642214.25        NaN   
2                       -1         0.999561    1819368.03        NaN   

   near_price  bid_price  bid_size  ask_price  ask_size  wap    target row_id  
0         NaN   0.999812  60651.50   1.000026   8493.03  1.0 -3.029704  0_0_0  
1         NaN   0.999896   3233.04   1.000660  20605.09  1.0 -5.519986  0_0_1  
2         NaN   0.999403  37956.00   1.000298  18995.00  1.0 -8.389950  0_0_2  
   stock_id  date_id  seconds_in_bucket  imbalance_size  \
0         0        0                  0      3180602.69   
1         1        0                

In [10]:
# for i in iter_test:
    
#     test_df = i[0]
#     print(test_df)
#     test_df['stats']  = pd.Series(test_df[stats_cols].fillna(-1).values.tolist())
#     stock_ids = test_df.stock_id.unique()
#     stocks = [trading_data.stocksDict[x] for x in stock_ids] 
#     hidden = torch.stack([trading_data.stocksDict[x].hidden for x in stock_ids]).transpose(0,1)
#     X = [torch.tensor(x) for x in test_df['stats'].tolist()]
#     X = torch.stack(X)
#     X = pack_sequence(X.view(-1,1,12))

    
#     model.eval()
#     output,hidden = model(X,hidden)
#     [setattr(obj, 'hidden', val.detach()) for obj, val in zip(stocks,hidden)]
#     print(output)
#     test_df['taget'] = output.flatten().tolist()
#     # test_df['actual'] = test_df['target']
#     env.predict(test_df)
    

In [11]:
output

NameError: name 'output' is not defined

In [347]:
x = test_df

In [356]:
test_df = x.head(5)

In [354]:
test_df

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,row_id,stats
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,60651.5,1.000026,8493.03,1.0,-3.029704,0_0_0,"[0.0, 3180602.69, 1.0, 0.999812, 13380276.64, ..."


In [596]:
for i,data in test_df.iterrows():
    hidden = torch.stack([trading_data.stocksDict[data['stock_id']].hidden])
    print(hidden)
    hidden = torch.stack([trading_data.stocksDict[data['stock_id']].hidden]).transpose(0,1)
    X = torch.tensor(data['stats']).view(-1,12,1)
    model.eval()
    output,hidden  = model(X,hidden,test=True)
    trading_data.stocksDict[data['stock_id']].hidden = hidden[0]
    print(hidden)
    print(output.flatten().item())

tensor([[[-0.8575,  0.7445, -0.8014,  0.7816, -0.5938, -0.7823, -0.7967,
          -0.5086,  0.5378, -0.0737, -0.5103, -0.5201, -0.2461, -0.4879,
          -0.3435, -0.1746, -0.1386,  0.0818, -0.7184, -0.3409, -0.6650,
           0.9997, -0.7376, -0.4719,  0.5599,  0.6262,  0.6931, -0.9579,
          -0.6052,  0.0718,  0.6607,  0.6729,  0.0845, -0.4968, -0.6447,
          -0.9867,  0.6466, -0.9892, -0.7533,  0.7768, -0.3308, -0.0147,
           0.9768,  0.5592,  0.0995, -0.1919,  0.9054,  0.9997, -0.0796,
          -0.7784, -0.9714, -0.4716, -0.5839,  0.9843, -0.7999,  0.9457,
           0.4040, -1.0000, -0.4622, -0.3323,  0.7707,  0.3389,  0.3091,
          -0.8781]]], grad_fn=<StackBackward0>)
tensor([[[-0.8575,  0.7445, -0.8015,  0.7816, -0.5938, -0.7823, -0.7968,
          -0.5089,  0.5378, -0.0737, -0.5103, -0.5201, -0.2461, -0.4878,
          -0.3442, -0.1746, -0.1390,  0.0821, -0.7184, -0.3408, -0.6650,
           0.9997, -0.7375, -0.4719,  0.5599,  0.6262,  0.6941, -0.9582,
   

In [350]:
test_df['stats']  = pd.Series(test_df[stats_cols].fillna(-1).values.tolist())
stock_ids = test_df.stock_id.unique()
hidden = torch.stack([trading_data.stocksDict[x].hidden for x in stock_ids]).transpose(0,1)
X = [torch.tensor(x) for x in test_df['stats'].tolist()]
# X = pack_sequence(X)

In [31]:
# X = torch.stack(X)

In [352]:
X = [torch.tensor(x) for x in test_df['stats'].tolist()]
X = torch.stack(X).view(-1,12,1)
# X = pack_sequence(X)
model.eval()
output,hidden  = model(X,hidden,test=True)
output

tensor([[[-2.1240],
         [ 1.0552],
         [-2.5545],
         [ 2.2515],
         [-1.9348],
         [ 3.7566],
         [-0.6949],
         [-2.2269],
         [-2.6248],
         [ 6.1143]]], grad_fn=<ViewBackward0>)

In [56]:
X.shape

torch.Size([1, 12, 10])

In [43]:
hidden

tensor([[[-0.1262, -0.1012, -0.2037,  0.1888,  0.0976, -0.2847, -0.5807,
          -0.5896, -0.0159,  0.2682, -0.5628,  0.1121, -0.1300,  0.2515,
           0.1864,  0.0224,  0.0196, -0.1425, -0.6340, -0.5149, -0.6020,
          -0.2689, -0.0297, -0.6029,  0.4218,  0.0321,  0.1283, -0.3335,
           0.2974, -0.4182,  0.1575,  0.3770, -0.1936, -0.5595,  0.2087,
          -0.1358,  0.1713,  0.1705,  0.0112, -0.2112, -0.5883, -0.2184,
           0.6552,  0.0014, -0.0707, -0.0010,  0.4840,  0.5455, -0.3142,
          -0.4542, -0.2142, -0.2478, -0.2938, -0.1563, -0.5573,  0.7317,
          -0.0501, -0.3461, -0.5378,  0.3539, -0.1586,  0.4133,  0.1239,
          -0.1093]]])

In [40]:
X

tensor([[[ 0.0000e+00,  3.1806e+06,  1.0000e+00,  9.9981e-01,  1.3380e+07,
          -1.0000e+00, -1.0000e+00,  9.9981e-01,  6.0652e+04,  1.0000e+00,
           8.4930e+03,  1.0000e+00]]])

In [ ]:
import importlib

In [ ]:
importlib.reload(torch_classes)
trading_data = torch_classes.TradingData(train)
hidden_size = 64
# trading_data.generate_batches()

In [ ]:
def model_pipeline(trading_df:torch_classes.TradingData, config:dict):
    with wandb.init(project="Optviver", config=config,save_code=True):
        wandb.define_metric("val_epoch_loss_l1", summary="min")
        wandb.define_metric("epoch_l1_loss", summary="min")
        model = torch_classes.GRUNet(12,hidden_size).to('cuda:0')
        config = wandb.config
        optimizer = optim.AdamW(model.parameters(), lr=config['learning_rate'])
        trading_df.reset_hidden(config['hidden_size'])
        criterion = nn.MSELoss()
        
        training_testing.train_model(trading_df,model,config,optimizer,criterion)

    return(model)



In [363]:
importlib.reload(training_testing)

<module 'training_testing' from 'c:\\Users\\Nick\\Documents\\GitHub\\OptiverKaggle\\training_testing.py'>

In [ ]:
config_static = {'learning_rate':0.0001, 'hidden_size':64}

In [ ]:
model = model_pipeline(trading_data, config_static)